In [1]:

from __future__ import print_function
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.datasets.data_utils import get_file
from keras.callbacks import Callback
import numpy as np
import random
import sys

import numpy as np
import csv
import matplotlib.pyplot as plt

synthetic_data_set = "syntheticDetailed/naive_c5_q50_s4000_v0.csv"

Using Theano backend.


In [4]:
# Read in the data set
# This function can be moved to utils.py
data_array = np.array(list(csv.reader(open(synthetic_data_set,"rb"),delimiter=','))).astype('int')
print (data_array[0:10])
num_samples = data_array.shape[0]
num_problems = data_array.shape[1]
num_timesteps = num_problems
# Split data into train and test (half and half)
train = data_array[0:num_samples/2,:]
test = data_array[num_samples/2:num_samples,:]
print (train.shape)
print (test.shape)

num_train = train.shape[0]
num_test = test.shape[0]


print('Vectorization...')
X_train = np.zeros((num_train, num_timesteps, num_problems * 2), dtype=np.bool)
y_train = np.zeros((num_train, num_timesteps, num_problems), dtype=np.bool)

# Create 3-dimensional input tensor with one-hot encodings for each sample
# the dimension of each vector for a student i and time t is 2 * num_problems
# where the first half corresponds to the correctly answered problems and the
# second half to the incorrectly answered ones.


for i in xrange(num_train):
    
    # for the first time step. Done separately so we can populate the output 
    # tensor at the same time, which is shifted back by 1.
#     if train[0,0] == 1:
#         X_train[0, 0, 0] = 1 # since timestep t corresponds to problem t
#     else:
#         X_train[0, 0, num_problems] = 1
            
    for t in xrange(0,num_timesteps):
        p = t # since timestep t corresponds to problem p where t=p
        if train[i,p] == 1:
            X_train[i, t, p] = 1 
        else:
            X_train[i, t, num_problems + p] = 1
#         y_train[i,t-1, p] = train[i,t,p]
    y_train[i,t,p] = train[i,p]
            
#     y_train[i,0:num_problems - 1] = train[i,1:num_problems]
#     y_train[i, num_problems-1] = 0





[[0 1 1 0 0 1 1 0 1 1 1 0 0 1 1 1 0 1 1 1 1 0 1 0 1 0 1 1 1 0 1 0 1 1 1 1 0
  0 1 1 0 1 1 0 1 0 1 1 0 1]
 [1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
  1 0 0 0 1 0 1 0 0 1 0 0 0]
 [0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 0 1 1 0 1 0 1 1 1 0 1 0 0 1 1 1 0 0 1
  1 1 1 0 0 1 1 1 0 0 1 1 1]
 [0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0 1 1 1 0 0 1 1
  0 1 0 0 1 1 1 1 0 0 1 1 1]
 [0 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 1
  1 1 1 0 0 0 1 0 1 1 1 1 1]
 [0 0 1 0 1 0 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1
  1 1 0 0 0 0 1 0 0 0 1 1 1]
 [0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 0 1 1 1 0 0 1 1 1 0 0 1 1 0 1 1 0 1
  1 1 1 0 0 1 1 1 1 1 1 1 1]
 [0 1 0 0 0 0 0 1 1 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1 1 0 0 1 1 0 0 1 0 1 1 0 0
  1 0 1 0 0 0 0 1 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 1
  1 0 1 0 0 0 1 0 1 1 0 1 0]
 [0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1

In [5]:
# build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(num_timesteps, num_problems * 2)))
model.add(Dropout(0.2))
model.add(LSTM(num_problems, return_sequences=True))
model.add(Dropout(0.2))
# model.add(Dense(num_problems, input_shape=(num_problems, num_problems)))
model.add(Activation('softmax'))

print('Compile model...')
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
print ('Compiling finished.')

Build model...
Compile model...

INFO (theano.gof.compilelock): Waiting for existing lock by process '27378' (I am process '32444')
INFO:theano.gof.compilelock:Waiting for existing lock by process '27378' (I am process '32444')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/lisa1010/.theano/compiledir_Darwin-15.2.0-x86_64-i386-64bit-i386-2.7.10-64/lock_dir
INFO:theano.gof.compilelock:To manually release the lock, delete /Users/lisa1010/.theano/compiledir_Darwin-15.2.0-x86_64-i386-64bit-i386-2.7.10-64/lock_dir



Compiling finished.


In [ ]:
def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

In [7]:
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

# train the model, output generated text after each iteration

epochs = 5
histories = [LossHistory() for i in xrange(epochs)]

for i in xrange(5):
    print()
    print('-' * 50)
    print('Iteration', i)
    model.fit(X_train, y_train, batch_size=200, nb_epoch=1, verbose=1, show_accuracy=True, callbacks=[histories[i]])
    




--------------------------------------------------
Iteration 0
Epoch 1/1
2000/2000 [==============================] - 44s - loss: 0.0527 - acc: 0.0201    

--------------------------------------------------
Iteration 1
Epoch 1/1
2000/2000 [==============================] - 44s - loss: 0.0462 - acc: 0.0154    

--------------------------------------------------
Iteration 2
Epoch 1/1
2000/2000 [==============================] - 40s - loss: 0.0437 - acc: 0.0137    

--------------------------------------------------
Iteration 3
Epoch 1/1
2000/2000 [==============================] - 40s - loss: 0.0430 - acc: 0.0137    

--------------------------------------------------
Iteration 4
Epoch 1/1
2000/2000 [==============================] - 45s - loss: 0.0423 - acc: 0.0138    


In [ ]:
losses = []
for h in histories:
    losses += h.losses
print losses

In [ ]:

plt.plot(xrange(len(losses)), losses)

# plot the trend line with error bars that correspond to standard deviation
plt.title('Loss History')
plt.xlabel('epoch')
plt.ylabel('loss')

plt.show()

In [ ]:
print model.summary()